In [2]:
import os
import sys

from os.path import join, pardir
sys.path.append(pardir)

from bids import BIDSLayout
from itertools import product, chain

from nipype import config
from nipype.pipeline.engine import Workflow, Node
from ica_wf import make_subject_ica_wf

220510-10:46:35,926 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.7.0


In [3]:
config.enable_debug_mode()

### Preproc & ICA Flow:

(For every dataset:)  
For every subject:  
For every run:  
For every task:  
For every space:  

      1. extract filenames of bold & mask file
      2. feed into preproc.py
      -> return ICs


In [4]:
def get_datapaths(bids_layout, subject, session, run, task, space, outdir='/LOCAL/jzerbe/temp_results/melodic'):
    """
    Extract mask and bold file paths for one subject for one task
    of one run for one type of space.
    
    Input: BIDSlayout, subject, ...  
    Output: bold_file, mask_file
    """
    # check if run and session are present
    run = None if run in ('0','00', '') else run
    session = None if session in ('0','00', '') else session
    # get paths
    bold_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='bold',
                        return_type='filename'
                        )
    bold_file = [i for i in bold_file if "AROMA" not in i]
    
    mask_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='mask',
                        return_type='filename'
                        )
    out_dir = join(outdir,
                   f'sub-{subject}',
                   f'sub-{subject}_ses-{session}_task-{task}_run-{run}_space-{space}-melodic.nii.gz')
    return bold_file, mask_file, out_dir

In [5]:
def read_paths(bids_layout, subject="all", session="all", run="all", task="all", space="all",
               outdir="/LOCAL/jzerbe/temp_results/melodic"):
    """
    Check if all data paths or only specific paths are asked for and give them back.
    
    Input: BIDSlayout
    Optional Input: subject, session, run, task, space
    Output: one file with all bold and mask file paths as tuples
    """
    if all([param == "all" for param in (subject, session, run, task, space)]):
        subject = bids_layout.get(return_type='id', target='subject', desc='preproc')
        session = bids_layout.get(return_type='id', target='session', desc='preproc')
        run = bids_layout.get(return_type='id', target='session', desc='preproc')
        task = bids_layout.get(return_type='id', target='task', desc='preproc')
        space = bids_layout.get(return_type='id', target='space', desc='preproc')
    else:
        subject = subject
        session = [session] # TODO: for many runs/sessions, check if pybids gives just a number or a list
        run = [run]
        task = [task]
        space = [space]
    
    # check if run and session are present
    session = '0' if session == [] else session
    run = '0' if run == [] else run
    
    # all combinations
    combinations = list(product(subject, session, run, task, space))
    boldfiles_nested, maskfiles_nested, outdirs = zip(*[
        get_datapaths(bids_layout, *params, outdir) for params in combinations
    ])
    boldfiles = [val for sublist in boldfiles_nested for val in sublist]
    maskfiles = [val for sublist in maskfiles_nested for val in sublist]
    
    return boldfiles, maskfiles, outdirs 

In [6]:
# create layout from BIDS dataset
bidsdata_dir = '/LOCAL/jzerbe/faces_vs_houses/ds002938'
layout = BIDSLayout(bidsdata_dir, derivatives=True)

In [7]:
# get all data paths
OUT_DIR = '/LOCAL/jzerbe/temp_results/melodic' # later change to /bidsdata_dir + /melodic
boldlist, masklist, outdirlist = read_paths(layout, outdir=OUT_DIR)

In [1]:
## testing: get data paths for one subject for certain parameters
#subjects = ['03'] 
#sessions = ['0'] # per subject
#runs = ['0']
#tasks = ['effort']
#spaces = ['T1w']

In [8]:
# parameter for melodic workflow
TR = 1.5
HPF = 120./TR
FWHM = 4.0
BASE_DIR = '/LOCAL/jzerbe/temp_results' # change to /bidsdata_dir?

In [9]:
# create a list with one workflow per bold/mask/output path
runwfs = []
i = 1
for boldfile, maskfile, outdir in zip(boldlist, masklist, outdirlist):
    print('ITERATION: ', i) # check if it really creates them all
    runwf = make_subject_ica_wf()
    
    runwf.inputs.inputspec.hpf = HPF
    runwf.inputs.inputspec.tr = TR
    runwf.inputs.inputspec.fwhm = FWHM
    runwf.base_dir = BASE_DIR    
    runwf.inputs.inputspec.bold_file = boldfile
    runwf.inputs.inputspec.mask_file = maskfile
    runwf.inputs.inputspec.out_dir = outdir
    runwfs.append(runwf)
    i += 1


ITERATION:  1
220510-10:47:41,175 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): No edge data
220510-10:47:41,176 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): new edge data: {'connect': [('bold_file', 'boldfile'), ('mask_file', 'maskfile')]}
220510-10:47:41,177 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220510-10:47:41,177 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220510-10:47:41,177 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220510-10:47:41,178 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220510-10:47:41,178 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220510-10:47:41,179 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [

220510-10:47:41,275 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.calcthresh): new edge data: {'connect': [('bold_file', 'boldfile'), ('mask_file', 'maskfile')]}
220510-10:47:41,276 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220510-10:47:41,276 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220510-10:47:41,277 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220510-10:47:41,277 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220510-10:47:41,278 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220510-10:47:41,278 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220510-10:47:41,279 nipype.workflow DEBUG:
	 (melod

220510-10:47:41,369 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): No edge data
220510-10:47:41,370 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220510-10:47:41,370 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220510-10:47:41,371 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220510-10:47:41,372 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220510-10:47:41,372 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220510-10:47:41,373 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220510-10:47:41,373 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_th

220510-10:47:41,489 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.susan): new edge data: {'connect': [('bold_file', 'in_file'), ('fwhm', 'fwhm')]}
220510-10:47:41,489 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220510-10:47:41,489 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220510-10:47:41,490 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220510-10:47:41,497 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220510-10:47:41,497 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220510-10:47:41,498 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_thresh', 'brightness_threshold')]}
220510-10:47:41,499 nipype.workflow DEBUG:
	 (melodicwf.susan, me

220510-10:47:41,577 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): No edge data
220510-10:47:41,578 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.tfilt): new edge data: {'connect': [('hpf', 'highpass_sigma')]}
220510-10:47:41,578 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): No edge data
220510-10:47:41,578 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220510-10:47:41,579 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220510-10:47:41,579 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_thresh', 'brightness_threshold')]}
220510-10:47:41,579 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220510-10:47:41,579 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'i

...

220510-10:47:41,857 nipype.workflow DEBUG:
	 (melodicwf.inputspec, melodicwf.melodic): new edge data: {'connect': [('mask_file', 'mask'), ('out_dir', 'out_dir'), ('tr', 'tr_sec')]}
220510-10:47:41,857 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220510-10:47:41,857 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_thresh', 'brightness_threshold')]}
220510-10:47:41,857 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220510-10:47:41,858 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'in_file')]}
220510-10:47:41,858 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220510-10:47:41,858 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220510-10:47:41,860 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
22051

220510-10:47:41,947 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): No edge data
220510-10:47:41,948 nipype.workflow DEBUG:
	 (melodicwf.calcthresh, melodicwf.susan): new edge data: {'connect': [('smooth_thresh', 'brightness_threshold')]}
220510-10:47:41,948 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220510-10:47:41,949 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'in_file')]}
220510-10:47:41,949 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220510-10:47:41,950 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220510-10:47:41,954 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220510-10:47:41,957 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220510-10:47:41,961 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220510-10:

220510-10:47:42,41 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): No edge data
220510-10:47:42,43 nipype.workflow DEBUG:
	 (melodicwf.susan, melodicwf.tfilt): new edge data: {'connect': [('smoothed_file', 'in_file')]}
220510-10:47:42,43 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220510-10:47:42,44 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220510-10:47:42,47 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220510-10:47:42,60 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220510-10:47:42,63 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220510-10:47:42,65 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-23/func/sub-23_task-effort_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220510-10:47:42,67 nipype.workflow DEBUG:
	 [No

220510-10:47:42,152 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): No edge data
220510-10:47:42,152 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220510-10:47:42,156 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220510-10:47:42,159 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220510-10:47:42,161 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220510-10:47:42,165 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-1back_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220510-10:47:42,168 nipype.workflow DEBUG:
	 [Node] inputspec - setting input mask_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-1back_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
220510-10:47:42,172 nipype.workflow DEBUG:
	 [Nod

220510-10:47:42,244 nipype.workflow DEBUG:
	 (melodicwf.tfilt, melodicwf.melodic): new edge data: {'connect': [('out_file', 'in_files')]}
220510-10:47:42,248 nipype.workflow DEBUG:
	 [Node] inputspec - setting input hpf = 80.0
220510-10:47:42,252 nipype.workflow DEBUG:
	 [Node] inputspec - setting input tr = 1.5
220510-10:47:42,255 nipype.workflow DEBUG:
	 [Node] inputspec - setting input fwhm = 4.0
220510-10:47:42,259 nipype.workflow DEBUG:
	 [Node] inputspec - setting input bold_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-effort_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz
220510-10:47:42,262 nipype.workflow DEBUG:
	 [Node] inputspec - setting input mask_file = /LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/fmriprep/sub-28/func/sub-28_task-effort_space-MNI152NLin6Asym_desc-brain_mask.nii.gz
220510-10:47:42,265 nipype.workflow DEBUG:
	 [Node] inputspec - setting input out_dir = /LOCAL/jzerbe/temp_results/melodic/sub-28/sub-28_ses-No

In [10]:
dataset_wf = Workflow(name='dataset_wf')
dataset_wf.add_nodes(runwfs)

OSError: Duplicate node name "melodicwf" found.